# Nettoyage & ETL - Accidents routiers

## Objectif
Fusionner les 3 tables en un dataset propre, avec :
- Une ligne par accident
- Une variable cible : accident mortel (oui/non)
- Des features exploitables pour le ML

In [1]:
import pandas as pd
import numpy as np

# Charger les données 2024
caract = pd.read_csv('../données/2024/caract-2024.csv', sep=';')
lieux = pd.read_csv('../données/2024/lieux-2024.csv', sep=';')
usagers = pd.read_csv('../données/2024/usagers-2024.csv', sep=';')

print(f"Caract: {caract.shape}, Lieux: {lieux.shape}, Usagers: {usagers.shape}")

Caract: (54402, 15), Lieux: (70248, 18), Usagers: (125187, 16)


## Stratégie de fusion

### Features (ce qu'on connaît AVANT l'accident)
- **CARACT** : conditions temporelles, météo, luminosité, localisation
- **LIEUX** : caractéristiques de la route

### Target (ce qu'on prédit)
- Accident mortel (oui/non) : agrégé depuis USAGERS

### Colonnes exclues (data leakage)
- `col` (type de collision) : résultat de l'accident
- **VEHICULES** : on ne sait pas quels véhicules seront impliqués
- **USAGERS** (sauf grav) : on ne sait pas qui sera impliqué

### Problèmes à traiter
1. Coordonnées GPS parfois invalides
2. Valeurs manquantes codées autrement que NaN
3. Date/heure éclatée sur plusieurs colonnes
4. Plusieurs lieux par accident

## Gestion des lieux multiples

Certains accidents ont plusieurs lieux (intersections). Il faut décider comment n'en garder qu'un.

In [2]:
# Accidents avec 2 lieux
accidents_2_lieux = lieux.groupby('Num_Acc').filter(lambda x: len(x) == 2)

# Prendre un exemple
exemple_acc = accidents_2_lieux['Num_Acc'].iloc[0]
print(f"Exemple accident {exemple_acc} :")
print(lieux[lieux['Num_Acc'] == exemple_acc][['Num_Acc', 'catr', 'voie', 'vma', 'surf']])

Exemple accident 202400000002 :
        Num_Acc  catr                    voie  vma  surf
1  202400000002     4  HOTEL DIEU (RUE DE L')   30     9
2  202400000002     4           POTERNE (RUE)   30     9


In [3]:
# Pour les accidents à 2 lieux, est-ce que vma et catr sont identiques ?
accidents_multi = lieux.groupby('Num_Acc').filter(lambda x: len(x) > 1)

# Comparer les valeurs par accident
def sont_identiques(group):
    return group['vma'].nunique() == 1 and group['catr'].nunique() == 1

resultats = accidents_multi.groupby('Num_Acc').apply(sont_identiques)
print(f"Accidents multi-lieux avec catr et vma identiques : {resultats.sum()} / {len(resultats)}")
print(f"Soit {resultats.mean()*100:.1f}%")

Accidents multi-lieux avec catr et vma identiques : 10934 / 15645
Soit 69.9%


### Observation

- 70% des accidents multi-lieux ont les mêmes `catr` et `vma`
- 30% ont des caractéristiques différentes entre les lieux

**Décision** : garder le premier lieu de chaque accident.
- Simple et reproductible
- Pour 70% des cas, ça ne change rien
- Pour 30%, c'est un choix arbitraire mais acceptable

Alternative possible : garder le lieu avec la `vma` la plus élevée (route principale). Mais ça complique sans garantie d'amélioration.

In [4]:
# Garder un seul lieu par accident (le premier)
lieux_unique = lieux.drop_duplicates(subset='Num_Acc')
print(f"Avant : {len(lieux)} lignes")
print(f"Après : {len(lieux_unique)} lignes")
print(f"Vérification : {lieux_unique['Num_Acc'].nunique()} accidents uniques")

Avant : 70248 lignes
Après : 54402 lignes
Vérification : 54402 accidents uniques


## Création de la variable cible

On agrège les usagers par accident : si au moins un usager a `grav == 2` (tué), l'accident est mortel.

In [5]:
# Créer la target : accident mortel (oui/non)
target = usagers.groupby('Num_Acc')['grav'].apply(lambda x: (x == 2).any())
target = target.astype(int)  # Convertir True/False en 1/0
target.name = 'mortel'

print(target.value_counts())
print(f"\nProportion mortels : {target.mean()*100:.1f}%")

mortel
0    51176
1     3226
Name: count, dtype: int64

Proportion mortels : 5.9%


## Fusion des tables

On fusionne :
- `caract` (circonstances)
- `lieux_unique` (1 lieu par accident)
- `target_df` (mortel oui/non)

Clé de jointure : `Num_Acc`

In [6]:
# Convertir target en DataFrame
target_df = target.reset_index()
target_df.columns = ['Num_Acc', 'mortel']

# Fusionner tout
df = caract.merge(lieux_unique, on='Num_Acc').merge(target_df, on='Num_Acc')

print(f"Dataset final : {df.shape}")
print(f"Accidents mortels : {df['mortel'].sum()}")

Dataset final : (54402, 33)
Accidents mortels : 3226


## Nettoyage

### Suppression des colonnes data leakage

In [ ]:
# Supprimer la colonne col (data leakage)
df = df.drop(columns=['col'])
print(f"Colonnes restantes : {df.shape[1]}")

In [8]:
# Explorer les valeurs manquantes
print(df.isnull().sum())

Num_Acc        0
jour           0
mois           0
an             0
hrmn           0
lum            0
dep            0
com            0
agg            0
int            0
atm            0
adr         2310
lat            0
long           0
catr           0
voie        7293
v1             0
v2         48760
circ           0
nbv            0
vosp           0
prof           0
pr             0
pr1            0
plan           0
lartpc     54374
larrout        0
surf           0
infra          0
situ           0
vma            0
mortel         0
dtype: int64


### Analyse des valeurs manquantes

Deux types de valeurs manquantes :
1. **NaN classiques** : `adr`, `voie`, `v2` (colonnes textuelles)
2. **-1 cachés** : dans les colonnes numériques (convention de la base BAAC)

In [9]:
# Chercher les -1 dans les colonnes numériques
colonnes_num = df.select_dtypes(include=[np.number]).columns
for col in colonnes_num:
    nb_moins1 = (df[col] == -1).sum()
    if nb_moins1 > 0:
        print(f"{col}: {nb_moins1} valeurs à -1")

v1: 11437 valeurs à -1
circ: 3524 valeurs à -1
vosp: 1280 valeurs à -1
prof: 11 valeurs à -1
plan: 8 valeurs à -1
surf: 6 valeurs à -1
infra: 568 valeurs à -1
situ: 5 valeurs à -1
vma: 1567 valeurs à -1


### Workflow de nettoyage

1. **Supprimer colonnes inutiles** : `adr`, `voie`, `v1`, `v2`, `pr`, `pr1`, `Num_Acc`, `com`
   - Texte brut non exploitable par le ML
   - On garde `lat`/`long` et `dep` pour la localisation

2. **Corriger les GPS** : format français (virgule → point) + filtrer métropole

3. **Garder -1 pour les catégorielles** : le modèle peut apprendre que "inconnu" a un pattern

4. **Imputer `vma`** si -1 : basé sur `catr` + `agg` (à faire)

In [10]:
# Colonnes à supprimer
cols_a_supprimer = ['adr', 'voie', 'v1', 'v2', 'pr', 'pr1', 'Num_Acc', 'com']

df = df.drop(columns=cols_a_supprimer)
print(f"Colonnes restantes : {df.shape[1]}")
print(df.columns.tolist())

Colonnes restantes : 24
['jour', 'mois', 'an', 'hrmn', 'lum', 'dep', 'agg', 'int', 'atm', 'lat', 'long', 'catr', 'circ', 'nbv', 'vosp', 'prof', 'plan', 'lartpc', 'larrout', 'surf', 'infra', 'situ', 'vma', 'mortel']


### Suppression des colonnes inutiles

In [11]:
# Vérifier les coordonnées GPS
# Convertir en float (format français : virgule → point)
df['lat'] = df['lat'].astype(str).str.replace(',', '.').astype(float)
df['long'] = df['long'].astype(str).str.replace(',', '.').astype(float)

print(f"Lat - min: {df['lat'].min()}, max: {df['lat'].max()}")
print(f"Long - min: {df['long'].min()}, max: {df['long'].max()}")

# Compter les valeurs hors France ou à 0
gps_invalides = (
    (df['lat'] == 0) |
    (df['long'] == 0) |
    (df['lat'] < 41) | (df['lat'] > 52) |
    (df['long'] < -6) | (df['long'] > 10)
)
print(f"\nGPS invalides : {gps_invalides.sum()} ({gps_invalides.mean()*100:.1f}%)")

Lat - min: -22.433239, max: 51.07874
Long - min: -178.094387, max: 167.863219

GPS invalides : 3347 (6.2%)


### Nettoyage des coordonnées GPS

Les coordonnées sont en format français (virgule décimale) et stockées comme texte.
Il faut les convertir en float.

In [12]:
# Vérifier le type actuel
print(f"Type lat : {df['lat'].dtype}")
print(f"Type long : {df['long'].dtype}")

Type lat : float64
Type long : float64


In [13]:
# Convertir lat/long en nombres (idempotent si déjà float)
df['lat'] = df['lat'].astype(str).str.replace(',', '.').astype(float)
df['long'] = df['long'].astype(str).str.replace(',', '.').astype(float)

print(f"Type lat : {df['lat'].dtype}")
print(f"Lat - min: {df['lat'].min():.2f}, max: {df['lat'].max():.2f}")
print(f"Long - min: {df['long'].min():.2f}, max: {df['long'].max():.2f}")

Type lat : float64
Lat - min: -22.43, max: 51.08
Long - min: -178.09, max: 167.86


### Filtrage France métropolitaine

Les données incluent les DOM-TOM (La Réunion, Polynésie, etc.) avec des conditions très différentes.

**Décision** : garder uniquement la métropole pour un modèle homogène.
- On perd ~6% des données
- Comportements routiers plus comparables
- V2 possible avec les DOM-TOM si besoin

In [14]:
# Filtrer France métropolitaine
metropole = (
    (df['lat'] >= 41) & (df['lat'] <= 51.5) &
    (df['long'] >= -5.5) & (df['long'] <= 10)
)
print(f"France métropolitaine : {metropole.sum()} ({metropole.mean()*100:.1f}%)")
print(f"DOM-TOM ou invalides : {(~metropole).sum()}")

France métropolitaine : 51055 (93.8%)
DOM-TOM ou invalides : 3347


In [15]:
# Garder uniquement la France métropolitaine
df = df[metropole].copy()
print(f"Dataset final : {df.shape}")

Dataset final : (51055, 24)


In [16]:
# Voir les vma par type de route et agglomération
print("VMA par catégorie de route (hors -1) :")
print(df[df['vma'] != -1].groupby(['catr', 'agg'])['vma'].median())

VMA par catégorie de route (hors -1) :
catr  agg
1     1      90.0
      2      70.0
2     1      90.0
      2      50.0
3     1      80.0
      2      50.0
4     1      70.0
      2      50.0
5     1      50.0
      2      30.0
6     1      30.0
      2      30.0
7     1      70.0
      2      50.0
9     1      50.0
      2      50.0
Name: vma, dtype: float64


### Imputation de vma (vitesse max autorisée)

Pour les `vma == -1`, on impute la médiane basée sur :
- `catr` : catégorie de route
- `agg` : en agglomération ou non

In [17]:
# Créer un dictionnaire de mapping
vma_mapping = df[df['vma'] != -1].groupby(['catr', 'agg'])['vma'].median().to_dict()
print("Mapping vma :")
print(vma_mapping)

# Imputer les -1
def imputer_vma(row):
    if row['vma'] == -1:
        return vma_mapping.get((row['catr'], row['agg']), 50)  # 50 par défaut
    return row['vma']

df['vma'] = df.apply(imputer_vma, axis=1)

# Vérifier
print(f"\nVMA -1 restants : {(df['vma'] == -1).sum()}")

Mapping vma :
{(1, 1): 90.0, (1, 2): 70.0, (2, 1): 90.0, (2, 2): 50.0, (3, 1): 80.0, (3, 2): 50.0, (4, 1): 70.0, (4, 2): 50.0, (5, 1): 50.0, (5, 2): 30.0, (6, 1): 30.0, (6, 2): 30.0, (7, 1): 70.0, (7, 2): 50.0, (9, 1): 50.0, (9, 2): 50.0}

VMA -1 restants : 0


### Suppression de lartpc

Colonne quasi-vide (28 valeurs sur 51055) - inutilisable.

In [18]:
# Vérifier lartpc
print(f"lartpc - valeurs non nulles : {df['lartpc'].notna().sum()}")

# Supprimer
df = df.drop(columns=['lartpc'])
print(f"Colonnes restantes : {df.shape[1]}")

lartpc - valeurs non nulles : 28
Colonnes restantes : 23


### Correction de nbv

La colonne `nbv` (nombre de voies) contient des valeurs `#VALEURMULTI` au lieu de nombres.
On les remplace par la médiane.

In [19]:
# Vérifier les valeurs invalides dans nbv
print(f"Valeurs uniques dans nbv : {df['nbv'].unique()[:20]}...")
print(f"Type actuel : {df['nbv'].dtype}")

# Compter les #VALEURMULTI
nb_invalides = (df['nbv'] == '#VALEURMULTI').sum()
print(f"\n#VALEURMULTI : {nb_invalides} occurrences")

# Corriger : remplacer par NaN puis par la médiane
df['nbv'] = df['nbv'].replace('#VALEURMULTI', np.nan)
df['nbv'] = df['nbv'].astype(float)
mediane_nbv = df['nbv'].median()
df['nbv'] = df['nbv'].fillna(mediane_nbv)

print(f"Médiane utilisée : {mediane_nbv}")
print(f"Type après correction : {df['nbv'].dtype}")

Valeurs uniques dans nbv : <StringArray>
[           '2',            '4',            '0',            '1',
          ' -1',            '3',            '6',            '8',
            '5',            '9', '#VALEURMULTI',           '10',
            '7',           '12',           '11']
Length: 15, dtype: str...
Type actuel : str

#VALEURMULTI : 44 occurrences
Médiane utilisée : 2.0
Type après correction : float64


### Correction de larrout

La colonne `larrout` (largeur de route) utilise le format français (virgule décimale).
On convertit en float.

In [20]:
# Convertir larrout : virgule → point
df['larrout'] = df['larrout'].astype(str).str.replace(',', '.').astype(float)

print(f"Type larrout : {df['larrout'].dtype}")
print(f"Exemples : {df['larrout'].head(5).tolist()}")

Type larrout : float64
Exemples : [7.0, -1.0, -1.0, -1.0, 17.0]


### Features temporelles

On crée des features plus exploitables à partir de la date/heure :
- `jour_semaine` : 0 (lundi) à 6 (dimanche)
- `is_weekend` : 1 si samedi/dimanche
- `heure` : 0 à 23

In [21]:
# Créer jour de la semaine (à partir de jour, mois, an)
df['date'] = pd.to_datetime(df[['an', 'mois', 'jour']].rename(columns={'an': 'year', 'mois': 'month', 'jour': 'day'}))
df['jour_semaine'] = df['date'].dt.dayofweek  # 0=lundi, 6=dimanche
df['is_weekend'] = df['jour_semaine'].isin([5, 6]).astype(int)

# Extraire l'heure de hrmn (format HHMM)
df['heure'] = df['hrmn'].astype(str).str.zfill(4).str[:2].astype(int)

# Supprimer les colonnes originales devenues inutiles
df = df.drop(columns=['jour', 'mois', 'an', 'hrmn', 'date'])

print(f"Colonnes : {df.shape[1]}")
print(df[['jour_semaine', 'is_weekend', 'heure']].head(10))

Colonnes : 22
   jour_semaine  is_weekend  heure
0             0           0      7
1             2           0     15
2             4           0     19
3             6           1     17
4             0           0     19
5             0           0      4
6             6           1      2
7             6           1     14
8             6           1      3
9             0           0     19


In [22]:
print(f"Dataset final : {df.shape}")
print(f"\nColonnes :")
print(df.columns.tolist())
print(f"\nTarget (mortel) :")
print(pd.Series(df['mortel']).value_counts())
print(f"\nProportion mortels : {df['mortel'].mean()*100:.1f}%")

Dataset final : (51055, 22)

Colonnes :
['lum', 'dep', 'agg', 'int', 'atm', 'lat', 'long', 'catr', 'circ', 'nbv', 'vosp', 'prof', 'plan', 'larrout', 'surf', 'infra', 'situ', 'vma', 'mortel', 'jour_semaine', 'is_weekend', 'heure']

Target (mortel) :
mortel
0    48044
1     3011
Name: count, dtype: int64

Proportion mortels : 5.9%


## Récap - Dataset nettoyé

| Étape | Résultat |
|-------|----------|
| Fusion tables | 54 402 accidents |
| Filtrage métropole | 51 055 accidents |
| Colonnes finales | 21 features + 1 target |
| Target | 5.9% mortels (déséquilibré) |

In [23]:
df.to_csv('../données/dataset_clean.csv', index=False)
print("Dataset sauvegardé : données/dataset_clean.csv")

Dataset sauvegardé : données/dataset_clean.csv
